# Notebook of Reinforcement Learning for Bandit Problems
This notebook was developed to explain a bandit problem, which is formally equivalent to a one-state Markov Decision Process (MDP). It is defined by a tuple $(\mathcal{A}, p)$, where $\mathcal{A}$ is a finite set of actions, and $p$ represents the probability for the reward given the current action. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from gym import Env, spaces, utils
from ipywidgets import interact, fixed
%matplotlib inline

# Example of the bandit problem
There are five slot machines, and therefore, an agent has five discrete actions. The agent receives a scalar reward according to the choice of actions. The goal is to find an action that maximizes the expected reward. 

In [ ]:
class GaussianBandit(Env):
    def __init__(self):
        self.n_machines = 5
        self.action_space = spaces.Discrete(self.n_machines)
        self.observation_space = spaces.Discrete(1)

        self.mu = np.array([1.0, 5.0, 3.0, 2.0, 3.5])
        self.sigma = np.array([0.8, 0.9, 1.5, 1.5, 1.2])

    def _step(self, action):
        done = True
        reward = np.random.normal(self.mu[action], self.sigma[action])
        return np.array([0]), reward, done, {}

    def _reset(self):
        return np.array([0])

    def plot(self):
        fig = plt.figure(figsize=(8,5))
        axarr = fig.subplots(1, 1)

        x = np.arange(start=-3.0, stop=8.0, step=0.05)
        for index, mean in enumerate(self.mu):
            std = self.sigma[index]
            norm_pdf = stats.norm.pdf(x=x, loc=mean, scale=std)
            axarr.plot(x, norm_pdf, label='r%1.0f' % index)
        axarr.legend()
        axarr.set_xlabel('r')
        axarr.set_ylabel('pdf')
        axarr.grid()

## Visualization of the reward functions

In [ ]:
env = GaussianBandit()
env.plot()

# Action selection
## $\varepsilon$-greedy policy
$\varepsilon$-greedy is a method to balance exploration and exploitation in the value-based reinforcement learning. When the currest estimate of the action-value function is $Q(a)$, the probability is to select the action $a$ is given by
\begin{equation}
  \pi (a) =
  \begin{cases}
    \epsilon/\lvert A \rvert + 1 - \epsilon & \mathrm{if} \; \;
       a = \mathrm{arg} \max_a Q(a), \\
    \epsilon/\lvert A \rvert & \mathrm{otherwise},
  \end{cases}
\end{equation}
where $\varepsilon \in [0, 1]$ is a hyperparameter and $\lvert A \rvert$ is the number of available actions. 

In [ ]:
def epsilon_greedy_policy(Q, epsilon=0.1):
    """Epsilon-greedy policy

    Parameters
    ----------
    Q : ndarray
        action-value function (1D array).
    epsilon : float
        hyperparameter to control the tradeoff between exploration and
        exploitation.
    
    Return
    ------
    policy : ndarray
        probability to select an action (1D array).
    """
    n_actions = Q.shape[0]
    policy = epsilon/n_actions*np.ones(n_actions)
    policy[Q.argmax(0)] += 1 - epsilon

    return policy


def plot_epsilon_greedy_policy(Q, epsilon=0.1):
    """Plot the epsilon-greedy policy

    Parameters
    ----------
    Q : ndarray
        action-value function (1D array).
    epsilon : float
        hyperparameter to control the tradeoff between exploration and
        exploitation.

    """
    policy = epsilon_greedy_policy(Q, epsilon)
    action = np.arange(Q.shape[0])

    fig = plt.figure(figsize=(12,5))
    axarr = fig.subplots(1, 2)
    axarr[0].bar(action, Q)
    axarr[0].set_xlabel('action a')
    axarr[0].set_ylabel('action value Q(a)')

    axarr[1].bar(action, policy, label=r'$\epsilon$ = %3.1f' % epsilon)
    axarr[1].set_xlabel('action a')
    axarr[1].set_ylabel(r'policy $\pi$ (a)')
    plt.legend()
    plt.show()

In [ ]:
# epsilon = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
Q = np.array([2.5, -2.5, 2.9, 1.2, 0.5])
interact(plot_epsilon_greedy_policy, Q=fixed(Q), epsilon=(0, 1, 0.05));

## Boltzmann distribution
The Boltzmann distribution is an alternative method for action selection. The probability distribution is defined by
\begin{equation}
  \pi (a) = \frac{ \exp (\beta Q(a)) }{\sum_{a'} \exp (\beta Q(a')) },
\end{equation}
where $\beta$ is a non-negative hyperparameter, which is often called the inverse temperature. 

In [ ]:
def Boltzmann_policy(Q, beta=1.0):
    """Boltzmann policy

    Parameters
    ----------
    Q : ndarray
        action-value function (1D array).
    beta : float
        hyperparameter to control the tradeoff between exploration and
        exploitation.

    """
    Qmax = np.max(Q)
    expQ = np.exp(beta * (Q - Qmax))
    policy = expQ / np.sum(expQ)

    return policy


def plot_Boltzmann_policy(Q, beta=1.0):
    """Plot the Boltzmann policy

    Parameters
    ----------
    Q : ndarray
        action-value function (1D array).
    beta : float
        hyperparameter to control the tradeoff between exploration and
        exploitation.

    """
    policy = Boltzmann_policy(Q, beta)
    action = np.arange(Q.shape[0])

    fig = plt.figure(figsize=(12,5))
    axarr = fig.subplots(1, 2)
    axarr[0].bar(action, Q)
    axarr[0].set_xlabel('action a')
    axarr[0].set_ylabel('action value Q(a)')

    axarr[1].bar(action, policy, label=r'$\beta$ = %3.1f' % beta)
    axarr[1].set_xlabel('action a')
    axarr[1].set_ylabel(r'policy $\pi$ (a)')
    plt.legend()
    plt.show()

In [ ]:
# beta = 1 #@param {type:"slider", min:0, max:5, step:0.2}
Q = np.array([2.5, -2.5, 2.9, 1.2, 0.5])
interact(plot_Boltzmann_policy, Q=fixed(Q), beta=(0, 8, 0.2));

# Value-based method

An agent learns an expected reward. When the agent selects an action $a$ and receives a scalar reward $r$, an estimate of the expected reward is updated by
\begin{equation*}
  Q(a) = Q(a) + \alpha (r - Q(a)),
\end{equation*}
where $\alpha$ is a learning rate. 

In [ ]:
import sys
def value_based_method(method, beta=1.0, epsilon=0.1, number_of_steps=1000):
    env = GaussianBandit()

    Q = np.zeros([env.action_space.n, number_of_steps+1])   # initialize an action value function
    N = np.zeros([env.action_space.n, number_of_steps+1])   # number of times "a" taken
    average_reward = 0

    for t in range(number_of_steps):
        if method == 'epsilon':
            policy = epsilon_greedy_policy(Q[:, t], epsilon=epsilon)
            action = np.random.choice(range(env.action_space.n), p=policy)
        elif method == 'Boltzmann': 
            policy = Boltzmann_policy(Q[:, t], beta=beta)
            action = np.random.choice(range(env.action_space.n), p=policy)
        else:
          sys.exit()  
        _, reward, _, _ = env._step(action)
        
        # compute the learning rate
        N[:, t+1] = N[:, t] + np.identity(env.action_space.n)[action]
        Q[:, t+1] = Q[:, t] # unselected actions
        alpha = 1/(N[action, t+1]) # learning rate
        # update the action value function
        Q[action,t+1] = Q[action,t] + alpha*(reward - Q[action,t])
        
        average_reward += reward
    
    # plot the learning curves of the action values
    average_reward /= number_of_steps
    fig = plt.figure(figsize=(12,5))
    axarr = fig.subplots(1, 2)
    for i in range(env.action_space.n):
        axarr[0].plot(Q[i, :], label='$a%i$' % i)
    axarr[0].legend()
    axarr[0].set_xlabel('steps')
    axarr[0].set_ylabel('action value Q(a)')
    axarr[0].grid()
    for i in range(env.action_space.n):
        axarr[1].plot(N[i, :], label='$N%i$' % i)
    axarr[1].legend()
    axarr[1].set_xlabel('steps')
    axarr[1].set_ylabel('N (a)')
    axarr[1].grid
    
    print(Q[:, -1])
    print('average reward: %f' % average_reward)

In [ ]:
epsilon = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
value_based_method('epsilon', epsilon=epsilon)

In [ ]:
beta = 1 #@param {type:"slider", min:0, max:5, step:0.2}
value_based_method('Boltzmann', beta=beta)

# Policy-based method
The policy-based method directly learns a stochastic policy. The basic idea is to utilize the gradient descent (ascent) method to update the parameters of the stochastic policy. 

## Gradient descent

Consider the minimization problem: $\min_x J(x)$, where $J(x)$ is differentiable. The local optimal solution is obtained by applying gradient descent:
\begin{equation}
x = x - \alpha \nabla_{x} J(x),
\end{equation}
where $\nabla_{x} J(x)$ is a partial derivative of $J(x)$ with respect to $x$, and $\alpha$ is a non-negative stepsize hyperparameter. 

In [ ]:
def eval_objective(x):
    Jx = 10*np.power(x, 2)
    dJx = 20*x
    return Jx, dJx

def optimize_by_gd(x_init, n_iterations, alpha):
    x = np.zeros(n_iterations+1)
    x[0] = x_init
    y = np.zeros(n_iterations+1)
    dy = np.zeros(n_iterations+1)
    for i in range(n_iterations):
        y[i], dy[i] = eval_objective(x[i])
        x[i+1] = x[i] - alpha*dy[i]
    y[-1], dy[-1] = eval_objective(x[-1])
    return x, y, dy


def plot_result(x, y, dy):
    xx = np.arange(-100, 100)
    Jx, _ = eval_objective(xx)
    
    fig = plt.figure(figsize=(12,5))
    axarr = fig.subplots(1, 2)
    axarr[0].plot(xx, Jx)
    axarr[0].plot(x, y, 'ro-')
    axarr[0].set_xlabel('x')
    axarr[0].set_ylabel('J(x)')
    axarr[1].plot(dy, 'ro-')
    axarr[1].set_xlabel('iteration')
    axarr[1].set_ylabel('dJ(x)/dx')

### Example of gradient descent
The following is the simple example of the gradient descent algorithm. The learning rate $\alpha$ affects convergence, and you can see that the algorithm never converges if $\alpha = 0.1$.

In [ ]:
alpha = 0.01 #@param {type:"slider", min:0.01, max:0.1, step:0.005}
x, y, dy = optimize_by_gd(x_init=-75, n_iterations=50, alpha=alpha)
plot_result(x, y, dy)

## Policy-based method
The Boltzmann distribution is chosen because the $\varepsilon$-greedy is not differentiable due to the max operator.

In [ ]:
def policy_based_method(method='reduction', alpha=0.05, number_of_steps=1000):
    env = GaussianBandit()

    theta = np.zeros([env.action_space.n, number_of_steps+1])   # initialize a policy parameter
    N = np.zeros([env.action_space.n, number_of_steps+1])   # number of times "a" taken
    average_reward = 0

    for t in range(number_of_steps):
        # compute a policy from the policy parameters
        policy = Boltzmann_policy(theta[:, t], beta=1.0)
        action = np.random.choice(range(env.action_space.n), p=policy)
        _, reward, _, _ = env._step(action)
        
        N[:, t+1] = N[:, t] + np.identity(env.action_space.n)[action]
        gradient = np.identity(env.action_space.n)[action] - policy
        if method == 'reduction':
            theta[:, t+1] = theta[:, t] + alpha*(reward - average_reward/(t+1))*gradient 
        elif method == 'no reduction':
            theta[:, t+1] = theta[:, t] + alpha*reward*gradient
        else:
            sys.exit()
        
        average_reward += reward
    
    # plot the learning curves of the action values
    average_reward /= number_of_steps
    fig = plt.figure(figsize=(12,5))
    axarr = fig.subplots(1, 2)
    for i in range(env.action_space.n):
        axarr[0].plot(theta[i, :], label='$a%i$' % i)
    axarr[0].legend()
    axarr[0].set_xlabel('steps')
    axarr[0].set_ylabel('policy parameter theta')
    axarr[0].grid()
    for i in range(env.action_space.n):
        axarr[1].plot(N[i, :], label='$N%i$' % i)
    axarr[1].legend()
    axarr[1].set_xlabel('steps')
    axarr[1].set_ylabel('N')
    axarr[1].grid

    print('average reward: %f' % average_reward)

In [ ]:
alpha = 0.03 #@param {type:"slider", min:0.01, max:0.1, step:0.005} 
policy_based_method('no reduction', alpha=alpha)

In [ ]:
alpha = 0.03 #@param {type:"slider", min:0.01, max:0.1, step:0.005} 
policy_based_method('reduction', alpha=alpha)